In [1]:
import numpy as np
import pandas as pd
import sklearn
import ot
import matplotlib.pyplot as plt


In [2]:
ratings_train = np.load('../datasets/ratings_train.npy')
ratings_test = np.load('../datasets/ratings_test.npy')
m,n = ratings_train.shape
data = np.nan_to_num(ratings_train)

movies_average_rating = np.nanmean(ratings_train, axis=0)
users_average_rating = np.nanmean(ratings_train, axis=1)
movies_average_rating_normalized = movies_average_rating / movies_average_rating.sum()
users_average_rating_normalized = users_average_rating / users_average_rating.sum()

movies_total_rating = np.nansum(ratings_train, axis=0)
users_total_rating = np.nansum(ratings_train, axis=1)
users_normalized_total_rating = users_total_rating / users_total_rating.sum()
movies_normalized_total_rating = movies_total_rating / movies_total_rating.sum()


In [88]:
def regularization(c,l):
    return l*np.power(np.linalg.norm(c),2)/2

def gradient_ridge_reg(c,c_hat,gamma,l):
    return l*c + (c - c_hat)/gamma



def compute_prox(c_hat, h_prox, x0, iterations = 100, gamma = 0.01, epsilon = 0.001,l = 0.01): #h is the non-differentiable function
    xk = x0
    gk = np.power(np.linalg.norm(xk-c_hat),2)/(2*gamma)
    for i in range(iterations):
        xk_old = xk
        # compute gradient for differentiable  part of function
        gk_gradient = gradient_ridge_reg(xk,c_hat,gamma,l)
        # take gradient step to reduce g(x)
        xk_gradient = xk - gamma * gk_gradient
        # proximal update to reduce h(x) but stay close to xk_gradient
        xk = xk_gradient #Change if h is constraining function

        if np.linalg.norm(xk - xk_old) < epsilon:
            print(f'we are here buddd in {i}')
            return xk

    return xk

def current_h(x,gamma):
    return 0


def cost_learning(data, pi_hat,mu,nu,epsilon,G, D, steps = 100):
    m,n = pi_hat.shape
    alpha = np.random.rand(m,1)
    beta = np.random.rand(n,1)
    u = np.exp(alpha / epsilon)
    v = np.exp(beta / epsilon)
    c = data
    
    G_inv = np.linalg.pinv(G)
    D_inv = np.linalg.pinv(D)

    A = np.random.rand(G_inv.shape[1], D_inv.shape[1])
    
    c_s = []
    
    for i in range(steps):
        K = np.exp(-c/epsilon)
        kv = np.dot(K,v).reshape(m,1)
        u = np.divide(mu.reshape(1,m),kv.reshape(1,m)).reshape(m,1)
        ktu = np.dot(K.T,u).reshape(n,1)
        v = np.divide(nu.reshape(1,n),ktu.reshape(1,n)).reshape(n,1)
        K = pi_hat / (np.dot(u,v.T))
        log_k = np.log(K, out=np.zeros_like(K), where=(K!=0))
        A_deomposed = G_inv.T@log_k@D_inv
        A = compute_prox(-epsilon*A_deomposed,current_h,A)
        c = G.T@A@D
        c = np.log(1 + np.exp(c))
        #c_s.append(c)
        #c = compute_prox(-epsilon*log_k,current_h,c)
        if np.linalg.norm(c - c_s[-1]) < epsilon:
            print(f'Cs equivalent in iteration {i}')


    return epsilon*np.log(u),epsilon*np.log(v),c


def transform_to_integers(matrix):
    num_intervals = 10
    min_value = matrix.min()
    max_value = matrix.max()

    # Calculate the interval width
    interval_width = (max_value - min_value) / num_intervals

    # Map the values to integers between 1 and 10
    transformed_matrix = ((matrix - min_value) / interval_width).clip(0, num_intervals).astype(int) / 2

    return transformed_matrix


def compute_rmse(predictions, test_matrix):
  masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
  diff = np.ma.subtract(predictions, masked)
  squared = np.ma.power(diff, 2)
  return np.ma.sqrt(np.ma.mean(squared))


In [4]:
from scipy.spatial.distance import pdist, squareform

user_dist = pdist(data, metric='jaccard')
user_sim_matrix = squareform(user_dist)

movie_dist = pdist(data.T, metric='jaccard')
movie_sim_matrix = squareform(movie_dist)


In [80]:
from sklearn.decomposition import NMF
import scipy.sparse as sps

csr_ratings_train = sps.csr_matrix(np.nan_to_num(ratings_train), shape=(610, 4980))
csr_ratings_train.eliminate_zeros()

nmf = NMF(n_components=10, init='nndsvd', random_state=0, max_iter=200)
W = nmf.fit_transform(csr_ratings_train)
H = nmf.components_


/Users/ellington/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [89]:
data = np.log(1 + np.exp(data))
pi_hat = data / data.sum()

mu_hat = pi_hat.sum(axis = 1)
nu_hat = pi_hat.sum(axis = 0)

G = W.T
D = H
# G = user_sim_matrix
# D = movie_sim_matrix

lambd = 1e-3
Gs = ot.sinkhorn(mu_hat, nu_hat, data, lambd)

epsilon = .1

alpha,beta,c = cost_learning(data, pi_hat, mu_hat, nu_hat, epsilon, G, D, steps=3)


/Users/ellington/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/ot/bregman.py:503: RuntimeWarning: divide by zero encountered in divide
  v = b / KtransposeU
/Users/ellington/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/ot/bregman.py:511: UserWarning: Warning: numerical errors at iteration 0
  warnings.warn('Warning: numerical errors at iteration %d' % ii)


we are here buddd in 1


IndexError: list index out of range

In [87]:
# otm = ot.emd(data.sum(axis=1), data.sum(axis=0), c)
otm_scores = transform_to_integers(c)
print(compute_rmse(otm_scores, ratings_train))
print(compute_rmse(otm_scores, ratings_test))


3.483452151033117
3.4740169704546675


In [32]:
np.unique(otm_scores, return_counts=True)


(610, 4980)